In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import time
from collections import Counter
import csv
import multiprocessing
from functools import partial
import math

In [4]:
import pandas as pd

# Replace with your CSV file name or path
file_path = "/content/review.csv"

# Read the CSV file
df = pd.read_csv(file_path)

# Display the first 10 entries
print(df.head(10))

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive


In [30]:
# parallel_text_analysis.py
import pandas as pd
import string
import time
import re
from collections import Counter
from multiprocessing import Pool, cpu_count
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# ------------------ Step 1: Load Dataset ------------------
dataset_path = '/content/review.csv'   # your CSV file name
df = pd.read_csv(dataset_path, nrows=20000)

# Auto-detect text column
text_col = next((c for c in df.columns if 'review' in c.lower() or 'text' in c.lower()), None)
if text_col is None:
    raise ValueError("No column containing text found in dataset!")

print(f"Using text column: {text_col}")

# ------------------ Step 2: Clean Text ------------------
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'<.*?>', ' ', text)     # remove HTML tags like <br/>
    text = text.translate(translator)      # remove punctuation
    tokens = text.split()
    return [w for w in tokens if w not in stop_words and w != 'br']

# ------------------ Step 3: Worker Function ------------------
def process_chunk(text_chunk):
    local_counter = Counter()
    for line in text_chunk:
        local_counter.update(clean_text(line))
    return local_counter

# ------------------ Step 4: Parallel Counting ------------------
def parallel_word_count(n_workers):
    chunks = []
    chunk_size = len(df) // n_workers
    for i in range(n_workers):
        start = i * chunk_size
        end = len(df) if i == n_workers - 1 else (i + 1) * chunk_size
        chunks.append(df[text_col][start:end])

    start = time.perf_counter()
    with Pool(processes=n_workers) as pool:
        results = pool.map(process_chunk, chunks)

    total_counter = Counter()
    for c in results:
        total_counter.update(c)

    elapsed = time.perf_counter() - start
    return total_counter, elapsed

# ------------------ Step 5: Run for 1, 2, 4, 8 workers ------------------
available = cpu_count()
print(f"\nDetected {available} CPU cores.")
baseline_time = None
summary = []

for n in [1, 2, 4, 8]:
    print(f"\nRunning with {n} worker(s)...")
    counter, elapsed = parallel_word_count(n)
    if baseline_time is None:
        baseline_time = elapsed
    speedup = baseline_time / elapsed
    efficiency = (speedup / n) * 100
    summary.append((n, elapsed, speedup, efficiency))
    print(f"Time: {elapsed:.2f}s | Speedup: {speedup:.2f}x | Efficiency: {efficiency:.1f}%")

# ------------------ Step 6: Save Top 20 Words ------------------
top_20 = counter.most_common(20)
pd.DataFrame(top_20, columns=['Word', 'Frequency']).to_csv('parallel_output.csv', index=False)

print("\n--- Performance Summary ---")
print("Workers | Time (s) | Speedup | Efficiency (%)")
print("----------------------------------------------")
for w, t, s, e in summary:
    print(f"{w:<8} | {t:<8.2f} | {s:<7.2f} | {e:<13.1f}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using text column: review

Detected 2 CPU cores.

Running with 1 worker(s)...
Time: 3.41s | Speedup: 1.00x | Efficiency: 100.0%

Running with 2 worker(s)...
Time: 2.23s | Speedup: 1.53x | Efficiency: 76.5%

Running with 4 worker(s)...
Time: 3.12s | Speedup: 1.09x | Efficiency: 27.3%

Running with 8 worker(s)...
Time: 2.60s | Speedup: 1.31x | Efficiency: 16.4%

--- Performance Summary ---
Workers | Time (s) | Speedup | Efficiency (%)
----------------------------------------------
1        | 3.41     | 1.00    | 100.0        
2        | 2.23     | 1.53    | 76.5         
4        | 3.12     | 1.09    | 27.3         
8        | 2.60     | 1.31    | 16.4         
